In [74]:
import random
import pandas as pd
import os

In [75]:
def generate_random_text(bits=80):
    # calculate the number of characters needed to represent 80 bits
    chars = bits // 8
    # generate a random string of alphanumeric characters
    text = "".join(random.choice(string.ascii_letters + string.digits)
                   for _ in range(chars))
    # return the text
    return text

In [76]:
def text_to_decimal_and_hexa(text):
    # encode the text as a bytes object using utf-8
    bytes = text.encode("utf-8")
    # convert the bytes object to an integer using int.from_bytes
    decimal_num = int.from_bytes(bytes, "big")
    hexa_num = hex(decimal_num)
    # return the number
    return decimal_num, hexa_num

In [77]:
def convert_to_text(number):
    # convert the number to a bytes object using int.to_bytes
    bytes = number.to_bytes((number.bit_length() + 7) // 8, "big")
    # decode the bytes object using utf-8
    text = bytes.decode("utf-8")
    # return the text
    return text

In [78]:
import string

# test the function
random_text = generate_random_text()
print(random_text)
decimal_num, hexa_num = text_to_decimal_and_hexa(random_text)
print(hexa_num)
print(decimal_num)
string = convert_to_text(decimal_num)
print(string)

CsX5Rtp7Is
0x43735835527470374973
318526285997851983432051
CsX5Rtp7Is


In [79]:
# import string

# # Specify the file path
# file_path = 'output.csv'

# # Clear the file content first
# with open(file_path, "w") as file:
#     file.write("")

# # generate 100 random texts
# for i in range(100):
#     random_text = generate_random_text()
#     # add each random text to a line in the file
#     with open(file_path, "a") as file:
#         file.write(random_text + "\n")

# # remove the last newline character
# with open(file_path, "rb+") as file:
#     file.seek(-1, 2)
#     file.truncate()

In [80]:
import string

file_path = 'output.csv'


# Define the number of rows you want in your DataFrame
num_rows = 1000

# Generate random texts using your function
texts = [generate_random_text() for _ in range(num_rows)]

# Generate random odd degrees between 1 and 13 (inclusive)
degrees = [random.randrange(1, 14, 2) for _ in range(num_rows)]

# Create a DataFrame with columns "text" and "degree"
df = pd.DataFrame({"random_text": texts, "degree": degrees})

In [81]:
df['degree'].value_counts()

degree
1     150
7     149
9     146
13    143
5     141
3     137
11    134
Name: count, dtype: int64

In [82]:
def generate_data_points(degree):
    """
    Generates a list of random (x, y) data points based on the given degree.
    """
    data_points = []
    for _ in range(degree + 1):
        # Generate random x and y coordinates here.
        x = random.random()
        y = random.random()
        data_points.append((x, y))
    return data_points

In [83]:
df["data_points"] = df["degree"].apply(generate_data_points)

## Apply Lagrange to get the polynomial


In [84]:
from scipy.interpolate import lagrange
import numpy as np

def interpolate(data_points):
    """
    Returns a function that interpolates between the given data points.
    """
    # Extract the x and y coordinates from the data points
    x_coords, y_coords = zip(*data_points)
    # Apply lagrange interpolation to the data points
    polynomial = lagrange(x_coords, y_coords)
    # Return the polynomial function coefficients as a list
    return list(polynomial.coefficients)

In [85]:
# Apply the interpolate function to the data points
df["coeffs"] = df["data_points"].apply(interpolate)

In [98]:
# df['x_coords'], df['y_coords'] = zip(*df['data_points'])

In [86]:
def nth_root(deg):
    if 1 <= deg < 10:
        return deg / 2
    elif 10 <= deg < 15:
        return deg / 3
    else:
        return deg

In [87]:
df["nth_root"] = df["degree"].apply(nth_root)

In [88]:
df["text_decimal"], df["text_hexa"] = zip(
    *df["random_text"].apply(text_to_decimal_and_hexa))

In [89]:
# Assuming your dataframe is named df
df['coeffs_min_text'] = df.apply(lambda row: [coeff - row['text_decimal'] if i == len(
    row['coeffs']) - 1 else coeff for i, coeff in enumerate(row['coeffs'])], axis=1)

In [90]:
from scipy.optimize import bisect, newton

tolerance = 1e-4
maxiter = 1500


def bisection_method(coeffs, a, b, tol=tolerance, maxiter=maxiter):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the bisection method using scipy's built-in function
    root = bisect(poly, a, b, xtol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)


def newton_method(coeffs, x0, tol=tolerance, maxiter=maxiter):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the newton method using scipy's built-in function
    root = newton(poly, x0, tol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)


def secant_method(coeffs, x0, x1, tol=tolerance, maxiter=maxiter):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the secant method using scipy's built-in function
    root = newton(poly, x0, x1=x1, tol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)

In [91]:
# # reorder the columns so coeffs_min_text comes after coeffs to better visualize the data
# df = df[['random_text', 'degree', 'data_points', 'coeffs', 'coeffs_min_text', 'nth_root', 'text_decimal', 'text_hexa']]

In [92]:
df.head(3)

,random_text,degree,data_points,coeffs,nth_root,text_decimal,text_hexa,coeffs_min_text
0,Z110YmGB3y,3,"[(0.5505791088494238, 0.8068143378968539), (0....","[-10.155250601494572, 6.380957612822016, 0.249...",1.5,425920418349112459670393,0x5a313130596d47423379,"[-10.155250601494572, 6.380957612822016, 0.249..."
1,0PFFR4EoBK,7,"[(0.5281795228139637, 0.5165248137177699), (0....","[-208202.01875450462, 688684.994969873, -92031...",3.5,228154394528855215784523,0x305046465234456f424b,"[-208202.01875450462, 688684.994969873, -92031..."
2,CP3Fvby07p,3,"[(0.25881471163116965, 0.016678245058483254), ...","[78.17576956705668, -139.51691401358755, 74.10...",1.5,317877988648872609593200,0x43503346766279303770,"[78.17576956705668, -139.51691401358755, 74.10..."


In [93]:
# Define a function to extract the constant term of a polynomial
def constant_term(polynomial):
    return polynomial[-1]

# Define a function to calculate the nth_root of a number


def nth_root(number, n):
    return np.sign(number) * np.abs(number) ** (1 / n)


# Create a new column called constant_nth_root by applying the functions to the polynomial and nth_root columns
df["constant_nth_root"] = df.apply(lambda row: nth_root(
    constant_term(row["coeffs"]), row["nth_root"]), axis=1)

In [94]:
# Import math library
import math


def bisection_initial(number):
    return [-round(number), round(number)]


def secant_initial(number):
    return [math.floor(number), math.ceil(number)]


def newton_initial(number):
    return round(number)


# Create a new column called bisection_initial by applying the function to the constant_nth_root column
df["bisection_initial"] = df["constant_nth_root"].apply(bisection_initial)
df["secant_initial"] = df["constant_nth_root"].apply(secant_initial)
df["newton_initial"] = df["constant_nth_root"].apply(newton_initial)

In [95]:
# # Create a new column called bisection_root by applying the function to the coeffs_min_text and bisection_initial columns
# df["bisection_cypher"] = df.apply(lambda row: bisection_method(
#     row["coeffs_min_text"], row["bisection_initial"][0], row["bisection_initial"][1]), axis=1)

# # Create a new column called newton_root by applying the function to the coeffs_min_text and newton_initial columns
# df["newton_cypher"] = df.apply(lambda row: newton_method(
#     row["coeffs_min_text"], row["newton_initial"]), axis=1)

# Create a new column called secant_root by applying the function to the coeffs_min_text and secant_initial columns
df["secant_cypher"] = df.apply(lambda row: secant_method(
    row["coeffs_min_text"], row["secant_initial"][0], row["secant_initial"][1]), axis=1)

RuntimeError: Tolerance of 1.0 reached. Failed to converge after 1 iterations, value is 1.

In [ ]:
# Define a function to evaluate a polynomial at a given root
def poly_val(coeffs, root):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Evaluate the polynomial at the root
    val = poly(root)
    # Return the value as a float
    return float(val)

In [ ]:
# # Create a new column called poly_val_bisection_root by applying the function to the coeffs and bisection_root columns
# df["poly_val_bisection_cypher"] = df.apply(lambda row: poly_val(row["coeffs_min_text"], row["bisection_cypher"]), axis=1)

# Create a new column called poly_val_newton_root by applying the function to the coeffs and newton_root columns
df["poly_val_newton_cypher"] = df.apply(
    lambda row: poly_val(row["coeffs_min_text"], row["newton_cypher"]), axis=1)

# Create a new column called poly_val_secant_root by applying the function to the coeffs and secant_root columns
df["poly_val_secant_cypher"] = df.apply(
    lambda row: poly_val(row["coeffs_min_text"], row["secant_cypher"]), axis=1)

In [ ]:
df

,random_text,degree,data_points,coeffs,nth_root,constant_nth_root,bisection_initial,secant_initial,newton_initial,text_decimal,text_hexa,coeffs_min_text,newton_root,secant_root,poly_val_newton_root,poly_val_secant_root
0,Ct2F87Fnff,9,"[(0.7478848831983145, 0.5566104826812637), (0....","[-146462.55591746687, 655049.9037939323, -1219...",4.500000,1.415130,"[-1, 1]","[1, 2]",1,318541999309576859444838,0x437432463837466e6666,"[-146462.55591746687, 655049.9037939323, -1219...",0.983688,0.999999,-1.529716e-05,-2.756734e+00
1,qXKhaZPBmv,9,"[(0.3080252753229893, 0.8067691253558819), (0....","[-455857.9367656243, 2250984.397220134, -47548...",4.500000,2.747158,"[-3, 3]","[2, 3]",3,535256159742747293674870,0x71584b68615a50426d76,"[-455857.9367656243, 2250984.397220134, -47548...",0.960463,0.960465,-5.042861e-05,-2.040286e-04
2,sKlLrPCdHU,9,"[(0.025507377394043185, 0.32100325511302175), ...","[-386201.84551147925, 1631025.2332938118, -291...",4.500000,1.876279,"[-2, 2]","[1, 2]",2,544463455073480795768917,0x734b6c4c725043644855,"[-386201.84551147925, 1631025.2332938118, -291...",0.914972,0.999977,-3.047925e-06,-1.699880e+02
3,hqcahCUkhF,1,"[(0.9733620170986094, 0.28025728305502784), (0...","[-0.22203273151279224, 0.4963755104622332]",0.500000,0.246389,"[0, 0]","[0, 1]",0,493217757418293182556230,0x687163616843556b6846,"[-0.22203273151279224, -4.9321775741829316e+23]",2.235596,2.235596,0.000000e+00,0.000000e+00
4,lvmWiejnRg,7,"[(0.43367965829704647, 0.46209835142559696), (...","[75624.46499388378, -250944.09717616794, 33684...",3.500000,-3.175743,"[3, -3]","[-4, -3]",-3,512200174832576816370279,0x6c766d5769656a6e5267,"[75624.46499388378, -250944.09717616794, 33684...",0.055263,0.055263,-7.516844e-05,-1.827582e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,f23Vv4ASzI,5,"[(0.2601195906339808, 0.20838537324848316), (0...","[2637.4369586015414, -6769.134431852652, 6648....",2.500000,-5.154591,"[5, -5]","[-6, -5]",-5,482607417730500027185737,0x66323356763441537a49,"[2637.4369586015414, -6769.134431852652, 6648....",0.245993,0.245993,-1.199965e-06,-6.481235e-08
996,Ul9vr3G0Os,7,"[(0.5814758993683026, 0.786376162806568), (0.1...","[320496.8566175868, -906598.0769334314, 104409...",3.500000,-4.349589,"[4, -4]","[-5, -4]",-4,403397540026352451735411,0x556c3976723347304f73,"[320496.8566175868, -906598.0769334314, 104409...",0.137811,0.137811,-1.414824e-05,-1.252951e-05
997,sV8qd1mKeg,3,"[(0.8977895227803699, 0.9027788986846838), (0....","[-59.920010134211864, 116.03206025303685, -61....",1.500000,3.262583,"[-3, 3]","[3, 4]",3,544662632662450163115367,0x7356387164316d4b6567,"[-59.920010134211864, 116.03206025303685, -61....",1.046699,1.046699,-6.710174e-08,-1.045259e-05
998,I53GFmDVuO,3,"[(0.6200808569727289, 0.7719841300802469), (0....","[1951.469286709471, -4378.2824352266, 3245.553...",1.500000,-85.713901,"[86, -86]","[-86, -85]",-86,345714125684845099513167,0x49353347466d4456754f,"[1951.469286709471, -4378.2824352266, 3245.553...",0.609751,0.609751,-1.444949e-07,-1.258107e-07


In [ ]:
df.to_csv(file_path, index=False, sep=';')